<a href="https://colab.research.google.com/github/ishika-thakur7802/medical-image-diagnosis/blob/main/major_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Add, Multiply, Activation, Reshape, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [3]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle

# Define CLAHE function
def apply_clahe(input_img):
    lab = cv2.cvtColor(input_img, cv2.COLOR_RGB2LAB)  # Convert to LAB color space
    l, a, b = cv2.split(lab)  # Split into Luminance, A, and B channels
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)  # Apply CLAHE on the Luminance channel
    lab = cv2.merge((cl, a, b))  # Merge channels back
    enhanced_img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)  # Convert back to RGB
    return enhanced_img

# Define image preprocessing function
def preprocess_image(image_path, label, use_clahe=True):
    input_img = cv2.imread(image_path)
    if input_img is None:
        print(f"Warning: Image {os.path.basename(image_path)} could not be loaded.")
        return None, None
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

    # Apply CLAHE if specified
    if use_clahe:
        input_img = apply_clahe(input_img)

    # Resize and normalize
    input_img_resize = cv2.resize(input_img, (224, 224))
    input_img_resize = input_img_resize / 255.0  # Normalize to [0, 1]
    return input_img_resize, label

# Define function to load and preprocess data from a folder
def load_data_from_folder(folder_path, label, use_clahe=True):
    img_data_list = []
    labels = []
    for dataset in os.listdir(folder_path):
        img, lbl = preprocess_image(os.path.join(folder_path, dataset), label, use_clahe)
        if img is not None:
            img_data_list.append(img)
            labels.append(lbl)
    return img_data_list, labels

# Load and preprocess dataset
data_path_0 = '/content/drive/MyDrive/major-project/OS Collected Data/Normal'  # Class 0
data_path_1 = '/content/drive/MyDrive/major-project/OS Collected Data/Osteopenia'  # Class 1
data_path_2 = '/content/drive/MyDrive/major-project/OS Collected Data/Osteoporosis'  # Class 2


# Load class 0 data
img_data_0, labels_0 = load_data_from_folder(data_path_0, label=0, use_clahe=True)

# Load class 1 data
img_data_1, labels_1 = load_data_from_folder(data_path_1, label=1, use_clahe=True)

# Load class 2 data
img_data_2, labels_2 = load_data_from_folder(data_path_2, label=2, use_clahe=True)

# Combine data and labels
img_data_list = img_data_0 + img_data_1+img_data_2
labels = labels_0 + labels_1+labels_2

# Convert to numpy arrays
X = np.array(img_data_list, dtype=np.float32)
y = np.array(labels)

# Shuffle data
X, y = shuffle(X, y, random_state=42)

# Print dataset summary
print(f"Total images: {X.shape[0]}")
print(f"Image shape: {X.shape[1:]}")
print(f"Labels distribution: {np.unique(y, return_counts=True)}")

Total images: 1947
Image shape: (224, 224, 3)
Labels distribution: (array([0, 1, 2]), array([780, 374, 793]))


In [4]:
print(f'Number of images: {len(img_data_list)}')
print(f'Number of labels: {len(labels)}')


Number of images: 1947
Number of labels: 1947


In [5]:
# Convert lists to arrays
img_data = np.array(img_data_list)
labels = np.array(labels)

print(f'Image data shape: {img_data.shape}')
print(f'Labels shape: {labels.shape}')


Image data shape: (1947, 224, 224, 3)
Labels shape: (1947,)


In [6]:
# One-hot encode the labels
labels = to_categorical(labels, num_classes=3)

In [7]:
print(f'Labels shape after one-hot encoding: {labels.shape}')

Labels shape after one-hot encoding: (1947, 3)


In [8]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(img_data, labels, test_size=0.2, random_state=42)

In [9]:
print(f'X_train shape after split: {X_train.shape}')
print(f'y_train shape after split: {y_train.shape}')
print(f'X_val shape after split: {X_val.shape}')
print(f'y_val shape after split: {y_val.shape}')

X_train shape after split: (1557, 224, 224, 3)
y_train shape after split: (1557, 3)
X_val shape after split: (390, 224, 224, 3)
y_val shape after split: (390, 3)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.applications import ResNet50

# Load a pre-trained ResNet model (excluding the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Build your model
model = Sequential([
    base_model,  # Pre-trained ResNet base model
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting
    Dense(3, activation='softmax')  # 3 classes (healthy, osteopenia, osteoporosis)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      51,380,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 74,969,987 (285.99 MB)

 Trainable params: 51,382,275 (196.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance for augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the generator on the training data
datagen.fit(X_train)


In [ ]:
# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=10,
                    validation_data=(X_val, y_val),
                    steps_per_epoch=len(X_train) // 32)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 54s 748ms/step - accuracy: 0.3957 - loss: 11.5997 - val_accuracy: 0.5462 - val_loss: 0.9032
Epoch 2/10
 1/48 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.4062 - loss: 1.0765

/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.4062 - loss: 1.0765 - val_accuracy: 0.6051 - val_loss: 0.8989
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step - accuracy: 0.5319 - loss: 1.0359

In [ ]:
# Unfreeze the top layers of the base model
base_model.trainable = True

# Freeze the first 10 layers and unfreeze the remaining layers
for layer in base_model.layers[:10]:
    layer.trainable = False

# Re-compile the model (required after modifying trainable layers)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
history_finetune = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                             epochs=10,
                             validation_data=(X_val, y_val),
                             steps_per_epoch=len(X_train) // 32)

In [ ]:
# Re-evaluate the model after fine-tuning
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy after fine-tuning: {test_acc:.4f}")
